# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [88]:
def line():
    print('-------------------------------------')

import pandas as pd
import numpy as np
from numpy import linalg
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Загрузка данных

In [89]:
data = pd.read_csv('/datasets/insurance.csv')
display(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [90]:
# Количество пустых строк
display(data.isnull().sum())
line()

# Количество дубликатов
display(data[data.duplicated()])
line()

# Общая информация по столбцам
data.drop_duplicates(inplace = True)
data.info()

data = data.reset_index()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

-------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


-------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Преобразованная формула:

$$
w' = ((PX)^T PX)^{-1} (PX)^T y 
$$

**Ответ:** Не изменится

**Обоснование:** 

$$
w' = (P^TX^TPX)^{-1}P^TX^Ty =
$$

$$
= (P^TX^TPX)^{-1}P^TX^Ty =
$$

$$
= (P^T)^{-1}P^{-1}(X^TX)^{-1}P^TX^Ty =
$$

$$
= EP^{-1}(X^TX)^{-1}(PX)^Ty =
$$

$$
= P^{-1}w
$$

Тогда:

$$
a' = PXw' = PXP^{-1}w = Xw = a
$$

Вывод: В принципе и без доказательства было понятно, что если мы домножим нашу матрицу признаков на обратную, то они в той же степени что и значения обратной матрицы пропорционально изменяться. Таким образом если мы рассмотрим пространство признаков с преобразованным пространством признаков, то все что изменится, это расстояние признаков, но относительно искомого значения мы не будем видеть разницы т.к хоть расстояние и изменится, но не изменится расстояние от ближайшего признака до цели

## Алгоритм преобразования

**Алгоритм**

Для того чтобы закодировать данные пользователя, можно создать рандомно генирирующуюся матрицу которая имеет обратную себе матрицу. Таким образом домножая матрицу признаков на рандомную матрицу матрицу мы получим матрицу с закодированными значениями а для того чтобы получить обратно данные мы должны будем умножить закодированные данные на обратную рандомную матрицу

**Обоснование**

Оно уже доказано в формулах выше

$$
a' = PXw' = PXP^{-1}w = Xw = a
$$

## Проверка алгоритма

In [93]:
# Получим матрицу признаков
data_matrix_features = data.drop(['Страховые выплаты'], axis=1).drop(['index'], axis = 1).values

# Цели
data_matrix_target = data['Страховые выплаты']

# Рандомная (P)
np.random.seed(1234)
random_matrix = np.random.rand(4,4)

# Рандомная (P^-1)
inv_random_matrix = linalg.inv(random_matrix)

PX = data_matrix_features @ random_matrix
PXP_mines_one = PX @ inv_random_matrix

In [94]:
print('Матрица признаков')
line()
display(pd.DataFrame(data_matrix_features))

print('Матрица признаков закодированная')
line()
display(pd.DataFrame(PX))

print('Матрица признаков раскодированная')
line()
display(pd.DataFrame(PXP_mines_one))

Матрица признаков
-------------------------------------


,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0
...,...,...,...,...
4842,0.0,28.0,35700.0,2.0
4843,0.0,34.0,52400.0,1.0
4844,0.0,20.0,33900.0,2.0
4845,1.0,22.0,32700.0,3.0


Матрица признаков закодированная
-------------------------------------


,0,1,2,3
0,47556.565933,43458.769791,17759.879603,24883.581540
1,36445.857790,33298.692082,13610.143865,19075.262086
2,20143.545726,18402.490515,7522.180133,10544.151929
3,39972.157466,36533.540718,14927.526408,20909.458442
4,25029.467973,22870.096468,9347.209473,13099.210556
...,...,...,...,...
4842,34228.781175,31279.853057,12782.558038,17909.100795
4843,50233.704773,45908.850911,18759.394981,26279.969428
4844,32497.890532,29700.993574,12136.275238,17000.894591
4845,31350.558241,28651.754408,11708.255421,16402.650740


Матрица признаков раскодированная
-------------------------------------


,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,1.224720e-11,46.0,38000.0,1.000000e+00
2,7.405031e-12,29.0,21000.0,-2.523768e-11
3,3.155380e-11,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-5.626201e-11
...,...,...,...,...
4842,6.834624e-12,28.0,35700.0,2.000000e+00
4843,3.898280e-11,34.0,52400.0,1.000000e+00
4844,1.289603e-11,20.0,33900.0,2.000000e+00
4845,1.000000e+00,22.0,32700.0,3.000000e+00


## Обучение регрессии

In [95]:
# Разделение выборок

features_train, features_test, target_train, target_test = train_test_split(
    data_matrix_features, data_matrix_target, test_size = 0.25, random_state = 12345)


features_train_cod, features_test_cod, target_train_cod, target_test_cod = train_test_split(
    PX, data_matrix_target, test_size = 0.25, random_state = 12345)

print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)
line()
print(features_train_cod.shape)
print(features_test_cod.shape)
print(target_train_cod.shape)
print(target_test_cod.shape)

(3635, 4)
(1212, 4)
(3635,)
(1212,)
-------------------------------------
(3635, 4)
(1212, 4)
(3635,)
(1212,)


In [101]:
# Обучение на обыных данных

model = LinearRegression()

model.fit(features_train, target_train)
predistions = model.predict(features_test)
print('R2_score на обычных данных:', r2_score(target_test, predistions))

# Обучение на закодированных данных

model.fit(features_train_cod, target_train_cod)
predistions = model.predict(features_test_cod)
print('R2_score на закодированных данных:', r2_score(target_test_cod, predistions))

R2_score на обычных данных: 0.42307727492147296
R2_score на закодированных данных: 0.4230772749239561


Вывод: Было доказано, что домножение признаков на матрицу которая имеет обратную себе ни как не повлияет на результат обучения линейной регрессии, а также может помоч в кодировании информации.